In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import find_peaks
import math
import os
from multiprocessing import Pool

In [68]:
# Real Cough Timestamps of the Data

def get_real_timestamps(cough_timestamp):

    timestamp = []
    f = open(cough_timestamp, "r")
    content = f.read()
    content = content.split("\n")

    for line in content:
        if line != "":
            timestamp.append(float(line.split("\t")[0]))
    
    return timestamp

In [69]:
# Moving Average of the Data

def compute_moving_average(data, window_size = 15):
    
    moving_averages = []

    i = 0
    while i < len(data) - window_size + 1:
        
        window = data[i : i + window_size]
        window_average = round(np.sum(window) / window_size, 2)
        moving_averages.append(window_average)

        i += 1    
    
    return np.array(moving_averages)

In [70]:
# Data Normalization

def normalize_data(data):
    min = np.min(data)
    max = np.max(data)

    data = (data - min) / (max - min)
    return data

In [71]:
# Cough Detection Algorithm

def detect_coughs(moving_avg_data):
    
    max_value = np.max(moving_avg_data)
    mean_value = np.mean(moving_avg_data)
    std = np.std(moving_avg_data)

    # Threshold 
    percentile_threshold = 99.8
    threshold = np.percentile(moving_avg_data, percentile_threshold)

    # Peak detection
    cough_indices, _ = find_peaks(moving_avg_data, prominence = 0.5)
    cough_indices = list(cough_indices)

    # Deleting overlaps in the peaks - Avoiding to count same cough more than one
    i = 0
    while i < len(cough_indices):

        peak = cough_indices[i]
        peak_range = (peak - 5000, peak + 5000) # The range is determined experimentally

        overlap_indices = [index for index in cough_indices
                       if peak_range[0] < index < peak_range[1]]

        if len(overlap_indices) > 1:

            # Find the index with maximum amplitude 
            max = overlap_indices[0]
            for index in overlap_indices:
                if (moving_avg_data[index] > moving_avg_data[max]):
                    max = index

            overlap_indices.remove(max)

            for element in overlap_indices:
                cough_indices.remove(element)

        i += 1


    # Applying the threshold
    cough_indices_copy = cough_indices.copy()
    for index in cough_indices_copy:

        amplitude = moving_avg_data[index]

        if (amplitude < threshold):
            cough_indices.remove(index)


    # Finding the timestamps of the coughs
    predicted_timestamps = []
    for index in cough_indices:
        predicted_timestamps.append(round(index / sample_rate, 6))


    # Filtering the sound after coughing
    for ts in predicted_timestamps:
        matches = list((ts_2 for ts_2 in predicted_timestamps if ts < ts_2 < ts + 0.4))

        if len(matches) != 0:
            for i in range(len(matches)):
                index = predicted_timestamps.index(matches[i])
                predicted_timestamps.remove(matches[i])
                cough_indices.remove(cough_indices[index])

    return cough_indices, predicted_timestamps

In [72]:
audio_files = os.listdir('./records')
timestamp_files = os.listdir('./timestamps')
all_timestamp_data = dict()

# Removing irrelevant files
for file in audio_files.copy():
    if not file.endswith('.wav'):
        audio_files.remove(file)

for file in timestamp_files.copy():
     if not file.endswith('.txt'):
        timestamp_files.remove(file)   

# Getting cough detection results
for file in audio_files:
        
    file_name = file.split(".wav")[0]    

    # Getting timestamp data of the specific file
    for timestamp_file in timestamp_files:
        
        timestamp_no = timestamp_file.split("-")[1]
        file_no = file_name.split("-")[1]
        
        if timestamp_no == file_no:
            timestamp_data = timestamp_file
            
    file_path = './records/' + file
    timestamp_path = './timestamps/' + timestamp_data
         

    print(file_path)
    print(timestamp_path)
    
    # Adding timestamps to the list
    real_timestamps = get_real_timestamps(timestamp_path)    
    
    # Loading audio_file
    # Target sample rate: 48000
    data, sample_rate = librosa.load(file_path, sr=48000)

    # Audio duration
    duration = librosa.get_duration(path = file_path)
    total_samples = duration * sample_rate
    time = np.arange(0, len(data)) / sample_rate

    # Getting moving average of the data
    moving_avg_data = compute_moving_average(np.abs(data))

    # Normalize the data
    normalized_moving_avg_data = normalize_data(moving_avg_data)
    
    # Cough detection
    cough_indices, predicted_timestamps = detect_coughs(normalized_moving_avg_data)

    # Adding predicted and real timestamp tuples to the list
    all_timestamp_data[file_name]=(predicted_timestamps, real_timestamps)

    cough_count = len(cough_indices)

    # The results
    print("\n{}".format(file_name))
    print("---------------------")
    print("Cough Count: {}".format(cough_count))
    print("Cough Predicted Timestamps: {}".format([round(timestamp, 6) for timestamp in predicted_timestamps]))
    print("Cough Real Timestamps: {}".format([round(timestamp, 6) for timestamp in real_timestamps]))
    print("Cough Indices: {}".format(cough_indices))

./records/audio-6.wav
./timestamps/audio-6-label.txt

audio-6
---------------------
Cough Count: 9
Cough Predicted Timestamps: [20.646417, 26.985646, 27.476583, 34.281833, 34.869125, 37.687812, 41.431312, 46.467375, 50.515604]
Cough Real Timestamps: [20.68898, 26.999365, 27.527256, 41.506032]
Cough Indices: [991028, 1295311, 1318876, 1645528, 1673718, 1809015, 1988703, 2230434, 2424749]
./records/audio-7.wav
./timestamps/audio-7-label.txt

audio-7
---------------------
Cough Count: 13
Cough Predicted Timestamps: [3.765646, 7.706438, 9.499188, 14.152375, 18.582542, 25.163729, 28.638, 37.638354, 39.257271, 39.884792, 43.039271, 47.368333, 52.808271]
Cough Real Timestamps: [7.767075, 25.205261, 47.403537]
Cough Indices: [180751, 369909, 455961, 679314, 891962, 1207859, 1374624, 1806641, 1884349, 1914470, 2065885, 2273680, 2534797]
./records/audio-5.wav
./timestamps/audio-5-label.txt

audio-5
---------------------
Cough Count: 3
Cough Predicted Timestamps: [7.280979, 32.6925, 49.692167]
Co

In [73]:
# Comparing the real timestamps and predicted timestamps.

def check_performance(predicted_timestamps, real_timestamps):
    time_margin = 0.05 # In seconds
    
    true_positive = 0 
    false_positive = 0

    if (len(predicted_timestamps) > len(real_timestamps)):

        for pred in predicted_timestamps:
            match = list((rt for rt in real_timestamps if pred - 0.4 < rt < pred + 0.4))

            if len(match) != 0:
                true_positive += 1
            else:
                false_positive += 1

    else:

        for pred in real_timestamps:
            match = list((rt for rt in predicted_timestamps if pred - 0.4 < rt < pred + 0.4))

            if len(match) != 0:
                true_positive += 1
            else:
                false_positive += 1
    
    return true_positive, false_positive

In [78]:
# General Performance of the Model
performances = []
true_positives = []
false_positives = []

for key in sorted(all_timestamp_data.keys()):
        
    data = all_timestamp_data.get(key)
    
    predicted_timestamps = data[0]
    real_timestamps = data[1]
    
    true_positive, false_positive = check_performance(predicted_timestamps, real_timestamps)
    
    true_positives.append(true_positive)
    false_positives.append(false_positive)
    
    precision = true_positive / (true_positive + false_positive)
    # Recall is assumed as 1 since the model does not predict the absence of coughs
    recall = 1

    f1_score = round(2 * (precision * recall) / (precision + recall), 3)
    print("{}: {}".format(key,f1_score))

    
model_performance = np.sum(true_positives) / np.sum(true_positives + false_positives)
print("\nModel Performance: ", model_performance)

audio-1: 1.0
audio-10: 1.0
audio-2: 0.933
audio-3: 1.0
audio-4: 1.0
audio-5: 1.0
audio-6: 0.615
audio-7: 0.375
audio-8: 0.0
audio-9: 1.0

Model Performance:  0.6461538461538462
